In [ ]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import re
import string
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

In [ ]:
!pip install transformers
from transformers import AutoTokenizer, AutoModel, AdamW
!pip install pytorch-pretrained-bert
from pytorch_pretrained_bert import BertModel

In [ ]:
# Get the GPU device name.
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

is_cuda = torch.cuda.is_available()

# If we have a GPU available, set our device to GPU. use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")   

Found GPU at: /device:GPU:0
GPU is available


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# data1 = pd.read_csv('/content/drive/My Drive/train_snli.txt', sep="\t", header=None)
# data1.columns = ["ans1", "ans2", "class"]
data = pd.read_csv('/content/drive/My Drive/Clean_train_data.csv')
# frames = [data1, data2]
# data = pd.concat(frames)

# Display 10 random rows from the data.
data.sample(10)

,Questions,ans1,ans2,class
2809,What do you understand by Perceptron?,Perceptron is a machine learning algorithm tha...,A Perceptron is an algorithm for supervised le...,1
2435,What is an RNN?,RNN) are the state of the art algorithm for se...,RNNs are designed to recognize the sequential ...,1
861,what do you know about output layer?,"The output layer is the simplest, usually cons...",Autoencoder is an artificial neural network. I...,0
2776,What do you understand by Perceptron?,A Perceptron is an algorithm used for supervis...,Perceptron is a machine learning algorithm tha...,1
3255,What are stop words?,Stop words are said to be useless data for a s...,Latent Semantic Indexing (LSI) tries to overco...,0
1943,Explain the following variant of Gradient Desc...,Mini-batch gradient descent is a variation of ...,a compromise between batch GD and SGD. In MB-G...,1
3607,Why is NLP so hard?,There are several factors that make the proces...,Information extraction (IE) is the automated r...,0
784,Explain Data Normalization.,The hidden layer after the input layer and bef...,change the values of numeric columns in the da...,0
1145,What is Model Capacity?,The Rectified Linear Unit is the most commonly...,The capacity of a deep learning neural network...,0
510,What is the meaning of term weight initializat...,"n neural networking, weight initialization is ...",This is a type of gradient descent which proce...,0


In [ ]:
# sentences_pairs = data[["sentence1", "sentence2"]].values.astype("str")
sents_pairs = data[["ans1", "ans2"]].values.astype("str")
labels = data["class"].values
print(sents_pairs.shape, labels.shape)

(4274, 2) (4274,)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
sents_pairs[0]

array(['Deep learning is a part of machine learning with an algorithm inspired by the structure and function of the brain, which is called an artificial neural network.Deep learning is suited over a range of fields such as computer vision, speech recognition, natural language processing, etc.',
       'Deep learning is a subset of machine learning in artificial intelligence that has networks capable of learning unsupervised from data that is unstructured or unlabeled. Also known as deep neural learning or deep neural network.'],
      dtype='<U1005')

In [ ]:
def clean_text(text):
    #Make text lowercase, remove text in square brackets,remove links,remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    return text

def clean2_text(text):
  #remove some of stopwords as 'a, an, the'
  txt = clean_text(text)
  words = txt.split(' ')
  aft_remove = [w for w in words if w not in ['a', 'an', 'the','of', 'that', 'which','in', 'at', 'for', 'where', 'when', 'to', 'from']]
  return ' '.join(aft_remove)

# p = clean_text(sents_pairs[0][0])
# print(sents_pairs[0][0])
# print(p)

# cl_tx = clean2_text(p)
# print(cl_tx)

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def parse_text(sentence):
  doc = nlp(sentence)
  # Analyze syntax
  nouns = [chunk.text for chunk in doc.noun_chunks]
  # print("Noun phrases:", nouns)

  #remove the nouns from sentence
  tmp = sentence
  for x in nouns:
    tmp = tmp.replace(x, '')
  # print(tmp)

  #analyze the verbs
  doc2 = nlp(tmp)
  verbs = [token.text for token in doc2 if token.dep_ == "VERB" or token.dep_ == "ROOT" or token.dep_ == "auxpass"]
  # print("Verbs:", verbs)
  # objects = [token.text for token in doc if token.dep_ == "iobj" or token.dep_ == "dobj" or token.dep_ == "obj" or token.dep_ == "nsubj"]

  tmp2 = tmp
  for x in verbs:
    # print(x)
    tmp2 = tmp2.replace(x, '')
  # print(tmp2)

  # print(len((' '.join(nouns)).split(' ')))
  if ( len( (' '.join(nouns)).split(' ') ) + len( (' '.join(tmp2)).split(' ') ) ) >= 2 and len(verbs) >= 1 :
    return True
  else:  
    return False

result = parse_text("data is made available output layer")
print(result)

True


In [ ]:
sentences_pairs = sents_pairs
for i, sent in enumerate(sents_pairs):
    tmp1 = str(sent[0])
    tmp2 = str(sent[1])
    if parse_text(tmp1) and parse_text(tmp2):
        sentences_pairs[i][0] = clean2_text(sent[0])
        sentences_pairs[i][1] = clean2_text(sent[1])
    else:
        print(tmp1)
        print(tmp2)
        print(False)
        sentences_pairs = np.delete(sentences_pairs, i)
        labels = np.delete(labels, i)
        # labels.pop(i)   

print(sentences_pairs.shape, labels.shape)

(4274, 2) (4274,)


In [ ]:
#know the maximum length of all sentences
def get_maximum(sent_pairs):
  max_len = 0
  # For every sentence...
  for sent in sent_pairs:
      # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
      input_ids = tokenizer.encode(sent[0], add_special_tokens=True)
      input_ids2 = tokenizer.encode(sent[1], add_special_tokens=True)
     
      max_len = max(max_len, len(input_ids) , len(input_ids2))
  return max_len

max_length = get_maximum(sentences_pairs)
print('Max sentence length: ', max_length)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.

def encoding_sentences(sentences_pairs, labels, max_length = 200):
  input_ids = []
  attention_masks = []
  token_type_ids = []

  # For every sentence...
  for sent_pair in sentences_pairs:

      encoded_pair = tokenizer(  sent_pair[0], sent_pair[1], 
                                padding= 'max_length',  # Pad to max_length
                                truncation= True,  # Truncate to max_length
                                max_length= max_length,  
                                return_tensors='pt'  
      )
      
       
      input_ids.append(encoded_pair['input_ids'])
      
      # And its attention mask that simply differentiates padding from non-padding.
      attention_masks.append(encoded_pair['attention_mask'])

      # Add its token type id to the list
      token_type_ids.append(encoded_pair['token_type_ids'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  token_type_ids = torch.cat(token_type_ids, dim=0)
  labels = torch.tensor(labels)

  return input_ids, attention_masks, token_type_ids, labels


input_ids, attention_masks, token_type_ids, labels = encoding_sentences(sentences_pairs, labels, 128)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences_pairs[0])
print('Token IDs:', input_ids[0])

Original:  ['deep learning is part machine learning with algorithm inspired by structure and function brain is called artificial neural networkdeep learning is suited over range fields such as computer vision speech recognition natural language processing etc'
 'deep learning is subset machine learning artificial intelligence has networks capable learning unsupervised data is unstructured or unlabeled also known as deep neural learning or deep neural network']
Token IDs: tensor([  101,  2784,  4083,  2003,  2112,  3698,  4083,  2007,  9896,  4427,
         2011,  3252,  1998,  3853,  4167,  2003,  2170,  7976, 15756,  2897,
        26095,  2361,  4083,  2003, 10897,  2058,  2846,  4249,  2107,  2004,
         3274,  4432,  4613,  5038,  3019,  2653,  6364,  4385,   102,  2784,
         4083,  2003, 16745,  3698,  4083,  7976,  4454,  2038,  6125,  5214,
         4083,  4895,  6342,  4842, 11365,  2098,  2951,  2003,  4895,  3367,
        26134,  2030,  4895, 20470, 12260,  2094,  2036,

In [ ]:
print(input_ids.shape, attention_masks.shape, labels.shape)


torch.Size([4274, 128]) torch.Size([4274, 128]) torch.Size([4274])


In [ ]:
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, token_type_ids, labels)

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))


3,419 training samples
  427 validation samples


In [ ]:
batch_size = 32

# take training samples in random order and Select batches randomly. 
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )

# For validation the order doesn't matter and  get batches sequentially.
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
        )

In [ ]:
class Similarity_Model(nn.Module):
    def __init__(self, output_dim, n_layers, hidden_dim, freeze_bert):
        super(Similarity_Model,self).__init__()
 
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.no_layers = no_layers
        
        #bert Model and Freeze the BERT model
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        if freeze_bert:
            for p in self.bert_model.parameters():
                p.requires_grad = False

        #LSTM layers
        # self.lstm = nn.LSTM(768, hidden_dim, n_layers, batch_first=True)

        # dropout layer
        self.dropout = nn.Dropout(0.3)

        # linear layer
        self.fc = nn.Linear(768, output_dim)
        self.sig = nn.Sigmoid()

    def forward(self, input_ids, attention_masks, token_type_ids, hidden):
        batch_size = input_ids.size(0)

        # print("batch" , batch_size)

        sequence_output, pooled_output = self.bert_model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids) 

        # print("seq  " , len(sequence_output))
        
        # lstm_out, hidden = self.lstm(sequence_output[0], hidden)
        
        # # print("lstm_out  " , lstm_out.shape)

        # lstm_out = lstm_out.permute(0,2,1)

        # # print("lstm_out  " , lstm_out.shape)

        
        # out_max = F.max_pool1d(lstm_out, kernel_size=lstm_out.shape[2])
        # out_avg = F.avg_pool1d(lstm_out, kernel_size=lstm_out.shape[2])
        
        # out = torch.cat((out_avg, out_max), dim=1)

        # # print("out  " , out.shape)

        # out = out.permute(0,2,1)

        # dropout and fully connected layer
        out = self.dropout(pooled_output)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        out = sig_out.view(batch_size, -1)

        out = out[:, -1]

        return out, hidden

    def init_hidden(self, batch_size):
        # initialize hidden states with sizes n_layers x batch_size x hidden_dim
        # initialized to zero, for hidden state and cell state of LSTM
        h0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        c0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        hidden = (h0,c0)
        return hidden

In [ ]:
no_layers = 1
output_dim = 1
hidden_dim = 128
Freeze_bert = False
model = Similarity_Model(output_dim, no_layers, hidden_dim, Freeze_bert)

#moving to gpu
model.to(device)

# model_save_name = 'Snli_TechSimilarity.pt'
# path = F"/content/drive/My Drive/{model_save_name}"
# model.load_state_dict(torch.load(path))
print(model)

Similarity_Model(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
# loss and optimization functions
lr=3e-5

criterion = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# function to predict accuracy
def acc(pred,label):
    pred = torch.round(pred.squeeze())
    return torch.sum(pred == label.squeeze()).item()

In [ ]:
def forward_back_prop(model, optimizer, criterion, batch, hidden):
    clip = 5
    # input_ids, attention_masks, token_type_ids = np.array(input_ids, dtype="int32"), np.array(attention_masks, dtype="int32"), np.array(token_type_ids, dtype="int32")
    # move data to GPU, if available
    input_ids, attention_masks, token_type_ids = batch[0].to(device), batch[1].to(device), batch[2].to(device)
    target = batch[3].to(device)
    h = tuple([each.data for each in hidden])

    optimizer.zero_grad()    
    output, hidden = model.forward(input_ids, attention_masks, token_type_ids, h)

    # calculate the loss and perform backprop
    loss = criterion(output.squeeze(), target.float())
    loss.backward()
    
    # calculating accuracy
    accuracy = acc(output,target)
    
    # clip_grad_norm prevent the exploding gradient problem in RNNs / LSTMs.
    nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model and accuracy
    return loss.item(), hidden, accuracy

In [ ]:
# model_save_name = 'Snli_TechSimilarity.pt'
# path = F"/content/drive/My Drive/{model_save_name}"

In [ ]:
epochs = 5
valid_loss_min = np.Inf

# train for some number of epochs
epoch_tr_loss,epoch_vl_loss = [],[]
epoch_tr_acc,epoch_vl_acc = [],[]

for epoch in range(1, epochs+1):
    train_losses = []
    train_acc = 0.0
    model.train()
    # initialize hidden state
    hidden = model.init_hidden(batch_size)

    for batch_i, batch in enumerate(train_dataloader, 1):

        # make sure iterate over completely full batches, only
        n_batches = len(train_indices)//batch_size
        if(batch_i > n_batches):
            break

        # forward, back prop
        loss, hidden, accuracy = forward_back_prop(model, optimizer, criterion, batch, hidden)  

        train_losses.append(loss)
        train_acc += accuracy
        
       
    val_h = model.init_hidden(batch_size)
    val_losses = []
    val_acc = 0.0
    model.eval()
    for batch_i, (input_ids, attention_masks,token_type_ids, labels) in enumerate(validation_dataloader, 1):  
        # make sure iterate over completely full batches, only
        n_batches = len(validation_dataloader.dataset)//batch_size
        if(batch_i > n_batches):
            break

        val_h = tuple([each.data for each in val_h])

        input_ids, labels = input_ids.to(device), labels.to(device)
        attention_masks,token_type_ids = attention_masks.to(device), token_type_ids.to(device)
        output, val_h = model(input_ids, attention_masks, token_type_ids, val_h)

        val_loss = criterion(output.squeeze(), labels.float())
        val_losses.append(val_loss.item())
        
        accuracy = acc(output,labels)
        val_acc += accuracy
            
    epoch_train_loss = np.mean(train_losses)
    epoch_val_loss = np.mean(val_losses)    
    epoch_train_acc = train_acc/len(train_dataloader.dataset)
    epoch_val_acc = val_acc/len(validation_dataloader.dataset)
    epoch_tr_loss.append(epoch_train_loss)
    epoch_vl_loss.append(epoch_val_loss)
    epoch_tr_acc.append(epoch_train_acc)
    epoch_vl_acc.append(epoch_val_acc)
    print(f'Epoch {epoch}') 
    print(f'train_loss : {epoch_train_loss} val_loss : {epoch_val_loss}')
    print(f'train_accuracy : {epoch_train_acc*100} val_accuracy : {epoch_val_acc*100}')
    if epoch_val_loss <= valid_loss_min:
        # torch.save(model.state_dict(), path)
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,epoch_val_loss))
        valid_loss_min = epoch_val_loss
    print(25*'==')

Epoch 1
train_loss : 0.2545150618929908 val_loss : 0.11424766036753471
train_accuracy : 89.29511553085698 val_accuracy : 93.6768149882904
Validation loss decreased (inf --> 0.114248).  Saving model ...
Epoch 2
train_loss : 0.11356116897198108 val_loss : 0.11334346535687263
train_accuracy : 95.9344837671834 val_accuracy : 93.20843091334895
Validation loss decreased (0.114248 --> 0.113343).  Saving model ...
Epoch 3
train_loss : 0.04866131750497756 val_loss : 0.10086795073360778
train_accuracy : 97.86487276981573 val_accuracy : 94.84777517564403
Validation loss decreased (0.113343 --> 0.100868).  Saving model ...
Epoch 4
train_loss : 0.02996311304796452 val_loss : 0.11726185109000653
train_accuracy : 98.39134249780638 val_accuracy : 95.08196721311475
Epoch 5
train_loss : 0.018662216824936278 val_loss : 0.14645663416908625
train_accuracy : 98.71307399824511 val_accuracy : 94.14519906323186


In [ ]:
# model.load_state_dict(torch.load(path))

In [ ]:
# predict the label given text
def predict_similrity(text, attention, token_ids):
        text, attention, token_ids = text.expand(1,-1), attention.expand(1,-1), token_ids.expand(1,-1)
        # print(text.shape, attention.shape, token_ids.shape, test_dataset[i][3].shape)

        inputs, attention, token_ids = text.to(device), attention.to(device), token_ids.to(device)
        batch_size = 1
        h = model.init_hidden(batch_size)
        h = tuple([each.data for each in h])
        output, h = model(inputs, attention, token_ids, h)
        return(output.item())

In [ ]:
#calculate test accuracy
sum = 0.0
for i in range(len(test_dataset)):
        # print(test_dataset[i][0].shape, test_dataset[i][1].shape, test_dataset[i][2].shape, test_dataset[i][3].shape)
        output = predict_similrity(test_dataset[i][0], test_dataset[i][1], test_dataset[i][2])
        if np.round(output) == test_dataset[i][3]:
            sum += 1

accuracy = sum/len(test_dataset)
print(f'test_accuracy : {accuracy*100}')    

test_accuracy : 97.66355140186917


In [ ]:
sentence1 = "First, Exploding gradient and this Solved by gradient clipping. Second, Dying ReLu — No learning if the activation is 0 (Solved by parametric relu). Third, Mean and variance of activations is not 0 and 1.(Partially solved by subtracting around 0.5 from activation. Better explained in fastai videos). "
sentence2 = "ReLU units can be fragile during training and can 'die'. For example, a large gradient flowing through a ReLU neuron could cause the weights to update in such a way that the neuron will never activate on any datapoint again. If this happens, then the gradient flowing through the unit will forever be zero from that point on. That is, the ReLU units can irreversibly die during training since they can get knocked off the data manifold."


encoded_pair = tokenizer(  sentence1, sentence2, 
                               padding= 'max_length',  # Pad to max_length
                               truncation= True,  # Truncate to max_length
                               max_length= 195,  
                               return_tensors='pt'  
    )

output = predict_similrity(encoded_pair['input_ids'], encoded_pair['attention_mask'], encoded_pair['token_type_ids'])

print(output)
print(np.round(output))

0.9980547428131104
1.0
